In [1]:
%%script cmd
az login --use-device-code

Microsoft Windows [Version 10.0.26200.6901]
(c) Microsoft Corporation. All rights reserved.

(aml-sandbox-env) q:\aml\tutorial-fine-tune\LLM-Finetuning\AzureML\1.SFT-LoRA>az login --use-device-code


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DC8YC37GD to authenticate.


[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72d2d42e-2b8a-4fa9-9e49-59eb283dc03f",
    "id": "ec782a34-0145-4786-b0e2-fc16ff1da279",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Visual Studio Enterprise Subscription",
    "state": "Enabled",
    "tenantDefaultDomain": "valitovroutlook.onmicrosoft.com",
    "tenantDisplayName": "Default Directory",
    "tenantId": "72d2d42e-2b8a-4fa9-9e49-59eb283dc03f",
    "user": {
      "name": "valitovr@outlook.com",
      "type": "user"
    }
  }
]

(aml-sandbox-env) q:\aml\tutorial-fine-tune\LLM-Finetuning\AzureML\1.SFT-LoRA>

In [2]:
!az account show

{
  "environmentName": "AzureCloud",
  "homeTenantId": "72d2d42e-2b8a-4fa9-9e49-59eb283dc03f",
  "id": "ec782a34-0145-4786-b0e2-fc16ff1da279",
  "isDefault": true,
  "managedByTenants": [],
  "name": "Visual Studio Enterprise Subscription",
  "state": "Enabled",
  "tenantDefaultDomain": "valitovroutlook.onmicrosoft.com",
  "tenantDisplayName": "Default Directory",
  "tenantId": "72d2d42e-2b8a-4fa9-9e49-59eb283dc03f",
  "user": {
    "name": "valitovr@outlook.com",
    "type": "user"
  }
}


In [4]:
# Import Libraries
import logging
import yaml
from azure.ai.ml import MLClient, Input, Output, command
from azure.ai.ml.entities import Environment, Model
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

In [5]:
import yaml
# read the Azure ML workspace configuration from config.yml
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

# Azure ML workspace configuration
subscription_id = config["subscription_id"]
resource_group = config["resource_group"]
workspace_name = config["workspace_name"]

In [6]:
registry_ml_client = MLClient(DefaultAzureCredential(), registry_name="azureml")

In [7]:
pipeline_component_func = registry_ml_client.components.get(
    name="chat_completion_pipeline", label="latest"
)
print(pipeline_component_func)

$schema: https://azuremlschemas.azureedge.net/latest/pipelineComponent.schema.json
name: chat_completion_pipeline
version: 0.0.80
display_name: Chat Completion Pipeline
description: Pipeline Component to finetune Hugging Face pretrained models for chat
  completion task. The component supports optimizations such as LoRA, Deepspeed and
  ONNXRuntime for performance enhancement. See [docs](https://aka.ms/azureml/components/chat_completion_pipeline)
  to learn more.
type: pipeline
inputs:
  pytorch_model_path:
    type: custom_model
    description: Pytorch model asset path. Special characters like \ and ' are invalid
      in the parameter value.
    optional: true
  mlflow_model_path:
    type: mlflow_model
    description: MLflow model asset path. Special characters like \ and ' are invalid
      in the parameter value.
    optional: true
  train_file_path:
    type: uri_file
    description: Path to the registered training data asset. The supported data formats
      are `jsonl`, `jso